In [1]:
##########################################################################################
############################### Final Design #############################################
## Final design with:
## Input: Word2vec vectors
## Output: One hot matrix
## Emotions: Positive 0, Neutral 1 and Negative 2
############################### start of code ############################################
import keras
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed, Merge
#from keras.utils.visualize_util import plot, to_graph
#import theano.d3viz as d3v
import word2vec_utils as w2v
import data
import numpy as np
from data_utils import split_dataset 

## Size of data
embed_dim = 300
data_dim = 1
timesteps = 26
vocab_size = 8100
emot_size = 4
max_sent_length = timesteps + emot_size

Using TensorFlow backend.


In [2]:
#Initialize Model
w2v_model = w2v.initialize()

In [8]:
## Convert input to word to vec matrix
metadata, filtered_A1, filtered_B, filtered_A2 = data.load_data()
# convert list of [lines of text] into list of [list of words ]


In [4]:
sm_data = 50000

In [5]:
print('>> w2v our inputs')
A1_w2v = np.array( [ w2v.vectorize( w2v_model, line, pad_length = max_sent_length ) for line in filtered_A1[sm_data:sm_data*2] ] ) 
B_w2v  = np.array( [ w2v.vectorize( w2v_model, line, pad_length = max_sent_length ) for line in filtered_B[sm_data:sm_data*2]  ] ) 
A2_w2v = np.array( [ w2v.vectorize( w2v_model, line, pad_length = max_sent_length ) for line in filtered_A2[sm_data:sm_data*2] ] ) 


>> w2v our inputs


/usr/local/lib/python2.7/dist-packages/gensim/models/keyedvectors.py:579: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return word in self.vocab


In [6]:
A1_train, A1_test, A1_val = split_dataset(A1_w2v)
B_train,  B_test,  B_val  = split_dataset(B_w2v )
A2_train, A2_test, A2_val = split_dataset(A2_w2v)

In [11]:
## Neural network model

keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)


In [29]:
## Emotional model
EM_Model = '''
model_Em = Sequential()
model_Em.add(LSTM(embed_dim,input_shape=(timesteps,embed_dim),return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model_Em.add(LSTM(embed_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model_Em.add(LSTM(embed_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model_Em.add(Dense(emot_size, activation='softmax'))

model_Em.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['accuracy']
              )

model_Em.fit(AB_train,AB_emtrain,
          batch_size=128, nb_epoch=20,
          validation_data=(AB_val,AB_emval))
model.save_weights('model_Em_weights.h5')
Em.predict(AB_train).concatenate(AB_train,axis=0)
'''

In [3]:
## model_A takes A's word2vec matrix as input 
model_A = Sequential()
model_A.add(LSTM(embed_dim,input_shape=(max_sent_length,embed_dim),return_sequences=True, dropout_W=0.2, dropout_U=0.2))

model_A.compile(loss='cosine_proximity',
              optimizer='rmsprop',
              metrics=['accuracy']
              )
#model_A.load_weights('model_A_weights.h5')
#model_A.fit(A1_train, A2_train,
#          batch_size=128, nb_epoch=5,
#          validation_data=(A1_val, A2_val))
#model_A.save_weights('model_A_weights.h5')

/Users/anthony/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, recurrent_dropout=0.2, dropout=0.2, input_shape=(30, 300), return_sequences=True)`
  app.launch_new_instance()


In [4]:
## model_B takes B's word2vec matrix as input 
model_B = Sequential()
model_B.add(LSTM(embed_dim,input_shape=(max_sent_length,embed_dim),return_sequences=True, dropout_W=0.2, dropout_U=0.2))

model_B.compile(loss='cosine_proximity',
              optimizer='rmsprop',
              metrics=['accuracy']
              )
#model_B.load_weights('model_B_weights.h5')
#model_B.fit(B_train, A2_train,
#          batch_size=128, nb_epoch=5,
#          validation_data=(B_val, A2_val))
#model_B.save_weights('model_B_weights.h5')

/Users/anthony/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, recurrent_dropout=0.2, dropout=0.2, input_shape=(30, 300), return_sequences=True)`
  app.launch_new_instance()


In [5]:
## model_ABA concatenates both A and B input and predicts next A
model_ABA = Sequential()
model_ABA.add(Merge([model_A, model_B], mode = 'concat'))
model_ABA.add(LSTM(embed_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model_ABA.add(LSTM(embed_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
#model_ABA.add(TimeDistributed(Dense(vocab_size,activation='softmax')))

model_ABA.compile(loss='cosine_proximity',
              optimizer='rmsprop',
              metrics=['accuracy']
              )
model_ABA.load_weights('model_ABA_weights.h5')
#graph = to_graph(model_ABA, show_shape=True)
#graph.write_png("model.png")
#d3v.d3viz(model_ABA.get_output(), 'test.html')
#model_ABA.fit([A1_w2v,B_w2v], A2_w2v,
#          batch_size=128, nb_epoch=50,
#          validation_split = 0.05 )
#          #validation_data=([A1_,B_val],A2_val))
#model_ABA.save_weights('model_ABA_weights.h5')

#score = model_ABA.evaluate([A1_test,B_test], A2_test, batch_size=64)

#print("\nModel Accuracy: %.2f%%" % (score[1]*100))

/Users/anthony/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()
/Users/anthony/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
/Users/anthony/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`


In [6]:
def cosign_similarity( a, b):
    dot = np.dot(a,b)
    norm = np.linalg.norm(a-b)
    return( dot/norm )

def most_likely_word( prediction, words, w2v_model ):
    ret = ""
    max_so_far = cosign_similarity( np.array([1]+[0]*299),prediction ) #must be better than blank
    #Best word in dictionary
    for word in words:
        similarity = cosign_similarity( w2v.vectorize(w2v_model,word)[0], prediction )
        if similarity > max_so_far:
            max_so_far = similarity
            ret = word    
    return( ret )

def most_likely_sent( prediction_vec, words, w2v_model ):
    predicted_words = []
    for prediction in prediction_vec:
        predicted_words.append( most_likely_word(prediction,words,w2v_model))
    ret = " ".join(predicted_words)
    return( ret )

In [10]:
User_A1 = "5 5 1 @I really love you. I think I want to marry you!"
User_B = "1 5 5 @yeah well I hate you and never want to see you again! "
User_A1 = w2v.vectorize( w2v_model, User_A1 , pad_length = max_sent_length) 
User_B  = w2v.vectorize( w2v_model, User_B, pad_length = max_sent_length) 
user_input = [ np.array( [User_A1]), np.array([User_B]) ]
prediction = model_ABA.predict(user_input)
most_likely_sent( prediction[0], metadata['w2idx'].keys(),w2v_model)

'1 9 1 @ yeah do you do just just                    '

In [73]:
cosign_similarity( prediction[0][-1], np.array([1]+[0]*299))

109.49282199122841

In [105]:
most_likely_sent(A2_train[0],metadata['w2idx'].keys(),w2v_model)

/Users/anthony/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in double_scalars


'1 9 1 @ forget it .                       '

In [110]:
prediction = model_ABA.predict([np.array([A1_train[0]]),np.array([B_train[0]])])
most_likely_sent(prediction[0],metadata['w2idx'].keys(),w2v_model)

'1 9 1 @ yeah guess you anyway just                     '

In [12]:
##########################################################################################
############################### 1) With Embedding ########################################

import keras
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed

import numpy as np
embed_dim = 128
data_dim = 1
timesteps = 25
vocab_size = 8100

x_old = np.load('idx_q.npy')
y_old = np.load('idx_a.npy')

keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

embedding = Sequential()
embedding.add(Embedding(vocab_size,embed_dim,input_length=timesteps))
embedding.compile('rmsprop', 'mse')

x = embedding.predict(x_old)
y = embedding.predict(y_old)

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
#model.add(Embedding(8100,embed_dim,input_length=timesteps))
model.add(LSTM(embed_dim,input_shape=(timesteps,embed_dim),return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(LSTM(embed_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(LSTM(embed_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
#model.add(TimeDistributed(Dense(1,activation='relu')))


model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['accuracy']
              )

# generate dummy training data


x_train = x[1:140001].reshape(140000,timesteps,embed_dim)
y_train = y[1:140001].reshape(140000,timesteps,embed_dim)
# generate dummy validation data
x_val = x[140001:150000].reshape(9999,timesteps,embed_dim)
y_val = y[140001:150000].reshape(9999,timesteps,embed_dim)
model.fit(x_train, y_train,
          batch_size=128, nb_epoch=20,
          validation_data=(x_val, y_val))
model.save('my_model.h5')
x_test = x[150001:160000].reshape(9999,timesteps,embed_dim)
y_test = y[150001:160000].reshape(9999,timesteps,embed_dim)
score = model.evaluate(x_test, y_test, batch_size=64)
print("\nModel Accuracy: %.2f%%" % (score[1]*100))

AttributeError: 'Sequential' object has no attribute 'load'

In [5]:
print(score)

[480337.9356212425, 0.012264528604810367]


In [2]:
c=model.predict(x[40001:40500].reshape(499,25))

In [3]:
print(c[1])

[ 0.09884203  0.08344298  0.09658517  0.09359372  0.08724034  0.07723846
  0.0680349   0.06117992  0.05216976  0.04656862  0.03937593  0.03422173
  0.03021572  0.02661941  0.02194737  0.01883822  0.01598314  0.0128208
  0.01082024  0.00868252  0.00629053  0.00474371  0.00292437  0.00145011
  0.00017013]


In [ ]:
##########################################################################################
############################ 2) Normal LSTM No Embedding #################################

import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed

import numpy as np
embed_dim = 128
data_dim = 1
timesteps = 25
vocab_size = 8100

x = np.load('idx_q.npy')
y = np.load('idx_a.npy')

keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

#embedding = Sequential()
#embedding.add(Embedding(vocab_size,embed_dim,input_length=timesteps))
#embedding.compile('rmsprop', 'mse')

#x = embedding.predict(x_old)
#y = embedding.predict(y_old)
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(data_dim,input_shape=(timesteps,data_dim),return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(LSTM(data_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(LSTM(data_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
#model.add(TimeDistributed(Dense(embed_dim,activation='relu')))
#model.add(TimeDistributed(Dense(data_dim,activation='relu')))
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['accuracy']
              )

# generate dummy training data


x_train = x[1:140001].reshape(140000,timesteps,data_dim)
y_train = y[1:140001].reshape(140000,timesteps,data_dim)
# generate dummy validation data
x_val = x[140001:150000].reshape(9999,timesteps,data_dim)
y_val = y[140001:150000].reshape(9999,timesteps,data_dim)
model.fit(x_train, y_train,
          batch_size=128, nb_epoch=20,
          validation_data=(x_val, y_val))
model.save('my_model.h5')
x_test = x[150001:160000].reshape(9999,timesteps,data_dim)
y_test = y[150001:160000].reshape(9999,timesteps,data_dim)
score = model.evaluate(x_test, y_test, batch_size=64)
print("\nModel Accuracy: %.2f%%" % (score[1]*100))

Using TensorFlow backend.


Train on 140000 samples, validate on 9999 samples
Epoch 1/20
140000/140000 [==============================] - 186s - loss: 402546.0464 - acc: 0.1487 - val_loss: 432623.3359 - val_acc: 0.0098
Epoch 2/20
140000/140000 [==============================] - 177s - loss: 402465.1430 - acc: 0.0107 - val_loss: 432610.5873 - val_acc: 0.0110
Epoch 3/20
 14208/140000 [==>...........................] - ETA: 149s - loss: 399385.0842 - acc: 0.0108

In [ ]:
##########################################################################################
################################# 3) LSTM One hot ########################################

import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed

import numpy as np
embed_dim = 128
data_dim = 1
timesteps = 25


x = np.load('idx_q.npy')
y_old = np.load('idx_a.npy')
vocab_size = np.amax(y_old)+1
y=(np.arange(y_old.max()) == y_old[:,:,None]-1).astype(int)
keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

#embedding = Sequential()
#embedding.add(Embedding(vocab_size,embed_dim,input_length=timesteps))
#embedding.compile('rmsprop', 'mse')

#x = embedding.predict(x_old)
#y = embedding.predict(y_old)
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(data_dim,input_shape=(timesteps,data_dim),return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(LSTM(data_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(LSTM(data_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(TimeDistributed(Dense(vocab_size,activation='softmax')))
#model.add(TimeDistributed(Dense(data_dim,activation='relu')))
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['accuracy']
              )

# generate dummy training data


x_train = x[1:140001].reshape(140000,timesteps,data_dim)
y_train = y[1:140001].reshape(140000,timesteps,vocab_size)
# generate dummy validation data
x_val = x[140001:150000].reshape(9999,timesteps,data_dim)
y_val = y[140001:150000].reshape(9999,timesteps,vocab_size)
model.fit(x_train, y_train,
          batch_size=128, nb_epoch=20,
          validation_data=(x_val, y_val))
model.save('my_model.h5')
x_test = x[150001:160000].reshape(9999,timesteps,data_dim)
y_test = y[150001:160000].reshape(9999,timesteps,vocab_size)
score = model.evaluate(x_test, y_test, batch_size=64)
print("\nModel Accuracy: %.2f%%" % (score[1]*100))

Using TensorFlow backend.


Train on 140000 samples, validate on 9999 samples
Epoch 1/20
 70272/140000 [==============>...............] - ETA: 141s - loss: 389574.3568 - acc: 0.0036

In [ ]:
##########################################################################################
################################# 4) LSTM Time distributed dense #########################
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed

import numpy as np
embed_dim = 128
data_dim = 1
timesteps = 25

keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
keras.optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.0)
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
#model.add(Embedding(8100,embed_dim,input_length=timesteps))
model.add(LSTM(data_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
model.add(LSTM(data_dim,return_sequences=True, dropout_W=0.2, dropout_U=0.2))
#model.add(LSTM(data_dim, dropout_W=0.2, dropout_U=0.2))
model.add(TimeDistributed(Dense(data_dim,activation='relu')))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy']
              )

# generate dummy training data
x = np.load('idx_q.npy')
y = np.load('idx_a.npy')

x_train = x[1:140001].reshape(140000,25)
y_train = y[1:140001].reshape(140000,25,1)
# generate dummy validation data
x_val = x[140001:150000].reshape(9999,25)
y_val = y[140001:150000].reshape(9999,25,1)
model.fit(x_train, y_train,
          batch_size=128, nb_epoch=20,
          validation_data=(x_val, y_val))
model.save('my_model.h5')
x_test = x[150001:160000].reshape(9999,25)
y_test = y[150001:160000].reshape(9999,25,1)
score = model.evaluate(x_test, y_test, batch_size=64)
print("\nModel Accuracy: %.2f%%" % (score[1]*100))

In [10]:
print(y_train.shape)

(25000,)


In [8]:
c = model.predict(X_test,batch_size=batch_size)

In [9]:
print(c[1])

[ 0.29208347]


In [14]:
import numpy as np
import tensorflow as tf
x = np.load('idx_q.npy')
y = np.load('idx_a.npy')
print(x.shape)
a=np.array([[1, 7, 5, 3], # Modified the sample listed in question for variety
       [2, 4, 1, 4]])

#b=(np.arange(y.max()) == y[:,:,None]-1).astype(int)
#b=np.eye(8100)[y]
print(np.amax(y))

(166559, 25)
8001
